

---


Vamos praticar!

Com o arquivo csv que você pesquisou, execute as seguintes atividades:

1 – Altere o nome de pelo menos cinco colunas

2 – Converta o tipo de dados de alguma coluna de sua escolha: Pode ser uma conversão para float, int ou date.

3 – Realize 5 consultas através da função pyspark.sql. Pelo menos duas das consultas deve conter alguma função de agregação: count(), sum(), avg(), etc.

4 – Com alguma das consultas da atividade 3 crie um novo DataFrame e salve-o em formato .csv e em formato .orc.


---



In [ ]:
#Criando ambiente Spark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=8527175209f2592e8821fdda72c9e281c352c2502534c1cdcb9153180f5ef59a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
#Importando funções e criando o objeto spark:
from pyspark.sql import SparkSession as ss
from pyspark.sql import functions as f

from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DecimalType

In [ ]:
spark = ss.builder.appName('SENAII').getOrCreate()

In [ ]:
spark

In [ ]:
url = '/content/drive/MyDrive/BasesSpark/GPlay/Google-Playstore.csv'
gplay = spark.read.csv(
        url,
        sep = ',',
        header = True,
        inferSchema = True
)

In [ ]:
gplay.show()

+----------------------------------+--------------------+-----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+------------+----------------+--------------+-------------------+
|                          App Name|              App Id|         Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|    Released|Last Updated|Content Rating|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+-----

In [ ]:
#1 – Altere o nome de pelo menos cinco colunas
gplay.withColumnsRenamed(
    {
       'App Name' :'nome_app',
       'App Id':'id',
       'Category' : 'categoria',
       'Rating':'avalia',
       'Rating Count': 'qtd_avalia',
       'Minimun Installs': 'downloads',
       'Price':'preco',
       'Released':'lancamento',
       'Developer':'Dev'
    }
).show()

+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|                          nome_app|                  id|        categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|           Developer|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+-------------

In [ ]:
#Efetivando alteração das colunas
gplay = gplay.withColumnsRenamed(
    {
       'App Name' :'nome_app',
       'App Id':'id',
       'Category' : 'categoria',
       'Rating':'avalia',
       'Rating Count': 'qtd_avalia',
       'Maximum Installs': 'downloads',
       'Price':'preco',
       'Released':'lancamento',
       'Content Rating':'classificacao',
       'Developer':'dev'
    }
)

In [ ]:
gplay.show()

+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|                          nome_app|                  id|        categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|                 dev|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+-------------

In [ ]:
gplay.printSchema()

root
 |-- nome_app: string (nullable = true)
 |-- id: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- avalia: string (nullable = true)
 |-- qtd_avalia: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Minimum Installs: string (nullable = true)
 |-- downloads: string (nullable = true)
 |-- Free: string (nullable = true)
 |-- preco: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Minimum Android: string (nullable = true)
 |-- Developer Id: string (nullable = true)
 |-- Developer Website: string (nullable = true)
 |-- Developer Email: string (nullable = true)
 |-- lancamento: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- classificacao: string (nullable = true)
 |-- Privacy Policy: string (nullable = true)
 |-- Ad Supported: string (nullable = true)
 |-- In App Purchases: string (nullable = true)
 |-- Editors Choice: string (nullable = true)
 |-- Scraped Time: s

In [ ]:
#2 – Converta o tipo de dados de alguma coluna de sua escolha: Pode ser uma conversão para float, int ou date.
#gplay.where('preco != "0"').show()
#Convertendo preço para double
gplay = gplay.withColumn('preco', gplay.preco.cast(DoubleType()))


In [ ]:
#Convertendo avaliação
gplay = gplay.withColumn('avalia', gplay.avalia.cast(DecimalType()))

In [ ]:
#Convertendo downloads
gplay = gplay.withColumn('downloads', gplay.downloads.cast(IntegerType()))

In [ ]:
#Criando view da consulta antes de alterar a data
gplay.createOrReplaceTempView('v_gplay1')

In [ ]:
#Convertendo data
gplay.withColumn('lancamento',\
                 f.to_date('lancamento','MMM d, yyyy')
                 ).show()

+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|                          nome_app|                  id|        categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+-------------

In [ ]:
#Verificando se a conversão de datas deu certo
spark.sql(
    """
    SELECT
      *
    FROM
      v_gplay1
    where id = 'com.imocci'
    limit 10
    """
).show()

+--------+----------+---------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+------------+--------------------+---------------+------------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|nome_app|        id|categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|Developer Id|   Developer Website|Developer Email|  lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+--------+----------+---------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+------------+--------------------+---------------+------------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|  IMOCCI|com.imocci|   Social|     0|         0|     50+|              50|       89|True| 

In [ ]:
#Efetivando conversão de data:
gplay = gplay.withColumn('lancamento',\
                 f.to_date('lancamento','MMM d, yyyy')
                 )

In [ ]:
#3 – Realize 5 consultas através da função pyspark.sql.
#Pelo menos duas das consultas deve conter alguma função de agregação: count(), sum(), avg(), etc.

#Criando nova view:
gplay.createOrReplaceTempView('v_gplay')

In [ ]:
#Testando view
spark.sql(
    """
    select * from v_gplay limit 10
    """
).show()

+--------------------+--------------------+----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|            nome_app|                  id|       categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+--------------------+--------------------+----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+-------

In [ ]:
gplay.limit(20).toPandas()

,nome_app,id,categoria,avalia,qtd_avalia,Installs,Minimum Installs,downloads,Free,preco,...,Developer Website,Developer Email,lancamento,Last Updated,classificacao,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0,0,10+,10,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,2020-02-26,"Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4,64,"5,000+",5000,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,2020-05-21,"May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0,0,50+,50,58,True,0.0,...,None,vnacrewit@gmail.com,2019-08-09,"Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5,5,10+,10,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,2018-09-10,"Oct 13, 2018",Everyone,None,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0,0,100+,100,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,2020-02-21,"Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35
5,IMOCCI,com.imocci,Social,0,0,50+,50,89,True,0.0,...,http://www.imocci.com,info@imocci.com,2018-12-24,"Dec 20, 2019",Teen,https://www.imocci.com/wp-content/uploads/2018...,False,True,False,2021-06-15 20:19:35
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,5,12,"1,000+",1000,2567,True,0.0,...,None,aitomgharfatimezzahra@gmail.com,2019-09-23,"Sep 27, 2019",Everyone,https://sites.google.com/view/unlimited4gdatap...,True,False,False,2021-06-15 20:19:35
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2,39,500+,500,702,True,0.0,...,None,elementuser03@gmail.com,2019-06-21,"Jun 21, 2019",Everyone,https://www.freeprivacypolicy.com/privacy/view...,False,False,False,2021-06-15 20:19:35
8,WhatsOpen,com.whatsopen.app,Communication,0,0,10+,10,18,True,0.0,...,http://yilvermolinah.com,yilver.mh1996@gmail.com,None,"Dec 07, 2018",Teen,http://elcafedelamanana.yilvermolinah.com/poli...,False,False,False,2021-06-15 20:19:35
9,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,5,820,"50,000+",50000,62433,True,0.0,...,https://trendyteme888-31139.web.app,trendyteme.888@gmail.com,2019-09-22,"Oct 07, 2020",Everyone,http://bit.ly/EmojiThemePro,True,False,False,2021-06-15 20:19:35




---


<h3>Atividade 3 </br>
Consulta 1 - Quantidade de aplicativos agrupados por notas

In [ ]:
spark.sql(
    """
    select
      case
        when avalia <= 1 then '0'
        when avalia >1 and avalia <=2 then '1'
        when avalia >2 and avalia <=3 then '2'
        when avalia >3 and avalia <=4 then '3'
        when avalia >4 and avalia <=5 then '4'
        when avalia = 5 then '5'
        else 'outros'
      end as Estrelas,
      count(*) as QTD_APP
    from
      v_gplay
    group by Estrelas
    order by 1
    """
).show()

+--------+-------+
|Estrelas|QTD_APP|
+--------+-------+
|       0|1062821|
|       1|  34172|
|       2| 154932|
|       3| 615103|
|       4| 423003|
|  outros|  22913|
+--------+-------+



---


<h3>Atividade 3 </br>
Consulta 2 - 100 Apps vendidos em Dólar que mais geraram lucro

In [ ]:
spark.sql(
    """
    select
      nome_app as Nome_App,
      downloads * preco as Receita
    from
      v_gplay
    where
      free = 'False'
      and currency = 'USD'
    order by 2 desc
    limit 100
    """
).toPandas()

,Nome_App,Receita
0,Minecraft,2.710131e+08
1,Poweramp Full Version Unlocker,2.735404e+07
2,OfficeSuite Pro + PDF,2.525882e+07
3,Grand Theft Auto: San Andreas,2.481188e+07
4,Hitman Sniper,2.470930e+07
...,...,...
95,Game Dev Story,4.044101e+06
96,NBA JAM by EA SPORTS™,4.013427e+06
97,Teach Your Monster to Read: Phonics & Reading ...,3.997324e+06
98,Kingdom Rush Origins - Tower Defense Game,3.972664e+06


---


<h3>Atividade 3 </br>
Consulta 3 - Top 10 desenvolvedores com mais aplicativos publicados

In [ ]:
gplay.show()

+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|                          nome_app|                  id|        categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|                 dev|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+-------------

In [ ]:
spark.sql( """
    select
      dev as Desenvolvedor,
      count(*) as Qtd_Apps
    from
      v_gplay
    group by dev
    order by Qtd_Apps desc
    limit 10

    """).show()

+--------------------+--------+
|       Desenvolvedor|Qtd_Apps|
+--------------------+--------+
|       Subsplash Inc|    5422|
|          TRAINERIZE|    5153|
|             ChowNow|    4865|
|           OrderYOYO|    2884|
|             Phorest|    2821|
|BH App Developmen...|    2453|
|          Sharefaith|    2077|
|            Flipdish|    1969|
|          J&M Studio|    1942|
|          CyJ Studio|    1741|
+--------------------+--------+



---


<h3>Atividade 3 </br>
Consulta 4 - Top 10 categorias de aplicativos mais baixados.

In [ ]:
gplay.show()

+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|                          nome_app|                  id|        categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|                 dev|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+-------------

In [ ]:
spark.sql( """
    select
      categoria,
      sum(downloads) Downloads
    from
      v_gplay
    group by categoria
    order by Downloads desc
    limit 10

    """).toPandas()

,categoria,Downloads
0,Tools,73532934161
1,Productivity,35688338257
2,Action,30742136749
3,Casual,30525269577
4,Entertainment,27939346236
5,Communication,27908388294
6,Photography,25836779557
7,Arcade,25681896317
8,Music & Audio,23021925013
9,Simulation,19957063168


---


<h3>Atividade 3 </br>
Consulta 5 - Média de dowsloads de aplicativos pagos por faixa de preço.

In [ ]:
gplay.show()

+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+--------------------+------------+----------------+--------------+-------------------+
|                          nome_app|                  id|        categoria|avalia|qtd_avalia|Installs|Minimum Installs|downloads|Free|preco|Currency|Size|Minimum Android|                 dev|   Developer Website|     Developer Email|lancamento|Last Updated|classificacao|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+----------+--------+----------------+---------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+----------+------------+-------------+-------------

In [ ]:
spark.sql(
    """
    (select
      'De 0 a 99 USD' as Preco,
      cast(AVG(downloads) as numeric(20,2)) as Downloads
    from
      v_gplay
    where
      preco between 0.01 and 99
      and categoria = 'Tools'
      and downloads > 0)
    union all
    (select
      'De 100 a 199 USD' as Preco,
      cast(AVG(downloads) as numeric(20,2)) as Downloads
    from
      v_gplay
    where
      preco between 100 and 199
      and categoria = 'Tools'
      and downloads > 0)
    union all
    (select
      'De 200 a 299 USD' as Preco,
      cast(AVG(downloads) as numeric(20,2)) as Downloads
    from
      v_gplay
    where
      preco between 200 and 299
      and categoria = 'Tools'
      and downloads > 0)
    union all
    (select
      'Acima de 300 USD' as Preco,
      cast(AVG(downloads) as numeric(20,2)) as Downloads
    from
      v_gplay
    where
      preco > 300
      and categoria = 'Tools'
      and downloads > 0)

    """
).show()

+----------------+---------+
|           Preco|Downloads|
+----------------+---------+
|   De 0 a 99 USD| 11522.61|
|De 100 a 199 USD|   163.75|
|De 200 a 299 USD|    98.75|
|Acima de 300 USD|   368.00|
+----------------+---------+



In [ ]:
app_lucro = spark.sql(
    """
    select
      nome_app as Nome_App,
      downloads * preco as Receita
    from
      v_gplay
    where
      free = 'False'
      and currency = 'USD'
    order by 2 desc
    limit 100
    """
)

In [ ]:
app_lucro.show()

+--------------------+--------------------+
|            Nome_App|             Receita|
+--------------------+--------------------+
|           Minecraft|2.7101308178000003E8|
|Poweramp Full Ver...|2.7354037290000003E7|
|OfficeSuite Pro +...|       2.525882427E7|
|Grand Theft Auto:...|       2.481187875E7|
|       Hitman Sniper|       2.470930407E7|
| Nova Launcher Prime|       2.440524669E7|
|Sleep as Android ...|       1.987227606E7|
|     Monument Valley|       1.805727168E7|
|[Legacy] Alarmy (...|       1.721544695E7|
|LockMyPix Photo V...|       1.702772829E7|
|            Terraria|       1.562141955E7|
|Docs To Go™ Premi...|1.4843466409999998E7|
|          True Skate|       1.401055122E7|
|         Bloons TD 6|       1.385978488E7|
|Torque Pro (OBD 2...|         1.3371039E7|
|Stickman Legends-...|       1.313181639E7|
|TuneIn Pro: Live ...|       1.237634127E7|
|         Bloons TD 5|1.2018672770000001E7|
|PJ Masks™: Racing...|       1.162333674E7|
|   Monument Valley 2|1.15138262

In [ ]:
#Salvando datraframe / view em um arquivo .orc (arquivo colunar)
url = '/content/drive/MyDrive/BasesSpark/ORC2'
app_lucro.write.orc(
    url,
    mode='overwrite'
)